In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

In [2]:
# check cuda devive
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Residual block implementation
As can be seen in the following diagram, there are two type of residual block 
1. w/o 1x1 conv
2. with 1x1 conv

![Alt text](images/residual_block.png)


In [3]:
# construct the residual block
# Residual block has multiple layers (Conv-BN-relu, shortcut connection)
# The key concept is the identity_downsample! Whenever connect to a new stage the identity needs to be downsampled
class residual_block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(residual_block, self).__init__()
        self.expansion = 4 # in teh 50/101/152, after the residual block the channel increase 4 times
        
        # the first 1x1 conv
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1   = nn.BatchNorm2d(out_channels) # batch_norm in the resnet paper is after the conv_layer, before the activation
        
        # the second conv kernel_size: 3x3, and with stide 2 -> you can observe the output side reduce is from the second conv
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)  # padding controls the amount of implicit zero-paddings on both sides for padding number of points for each dimension.
        self.bn2   = nn.BatchNorm2d(out_channels)
        
        # the third 1x1 conv, after the final layer, the channel size will increase 4 times
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3   = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample 
        
    def forward(self, x):
        identity = x.clone()
        
        # Stacking: Conv-BN-ReLU
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        # no relu here
   
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity) # use the nn.Sequential 
        
        x += identity # shortcut connection
        x = self.relu(x) # relu will be after the addition
        return x

In [4]:
class ResNet(nn.Module):
    def __init__(self, residual_block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels=64
        
        # Input image size (224, 224, 3)
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)  #padding 3!
        self.bn1   = nn.BatchNorm2d(64)
        self.relu  = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # padding1
        
        # resnet stage (multiple residual blocks -> multiple resnet layers) 
        self.stage1 = self._make_stage(residual_block, layers[0], 64, 1) # check the paper, the first stage is no down-sampling, because the previous maxpool already downsampleing
        self.stage2 = self._make_stage(residual_block, layers[1], 128, 2)
        self.stage3 = self._make_stage(residual_block, layers[2], 256, 2)
        self.stage4 = self._make_stage(residual_block, layers[3], 512, 2)
        
        self.avepool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4, num_classes)
    
    def _make_stage(self, residual_block, num_residual_blocks, out_channels, stride):
        '''
        Helper function to construct the stage of resnet
        * Resnet has multiple stage 
        * per stage has multiple residual_blocks
        * per residual_block has 3 multiple layers (conv-bn-relu, shortcut connection)
        output: nn.Sequential(*list of layers structure)
        '''
        identity_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != out_channels * 4:
            # stride != 1: whenever user want to down-sample using stride
            # self.in_channels != out_channels * 4: here is to deal with whenever connect to a new stage, the identity need to downsample!
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1, stride=stride), # make sure the depth dimension fits
                                                nn.BatchNorm2d(out_channels*4))
        
        # downsample need to be the beginning ?
        layers.append(residual_block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels*4
        
        for i in range(num_residual_blocks - 1):
            layers.append(residual_block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers) # unpack the layer
    
    def forward(self, x):
        
        # Initial standard step:
        x = self.conv1(x) # (224, 224, 3) -> (112, 112 , 64)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x) # (112, 112 , 64) -> (56, 56, 64)
        
        # stacking residual block
        x = self.stage1(x) # (56. 56, 64) -> (56, 56, 256)
        x = self.stage2(x) # (56, 56, 256) -> (28, 28, 512)
        x = self.stage3(x) # (28, 28, 512) -> (14, 14, 1024)
        x = self.stage4(x) # (14, 14, 1024) -> (7, 7, 2048)
                
        # average pool
        x = self.avepool(x) #(7, 7, 2024) -> (1, 1, 2048)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x) # 2048 * num_classes
        
        return x
        
        

In [5]:
# ResNet 50 will reuse the residual block [3,4,6,3], this will be the arg layers
ResNet50 = ResNet(residual_block, [3,4,6,3], image_channels=3, num_classes=1000)

In [6]:
ResNet101 = ResNet(residual_block, [3,4,23,3], image_channels=3, num_classes=1000)

In [7]:
ResNet152 = ResNet(residual_block, [3,8,36,3], image_channels=3, num_classes=1000)

In [8]:
x = torch.randn(4,3,224,224)

In [9]:
y = ResNet50(x).to(device)